In this notebook, I implemented Boyer-Moore and Approximate matching algorithms for sequence (DNA) alightment problems. There differences noted and efficiency compared.  

- Naive exact matching algorithms compare both alignments and characters from left to right
- Boyer-Moore is built upon naive exact matching, also compares alignments from left to right, but characters from right to left. This allows skipping unnecessary comparisons thus can be more efficient.
- Approximate matching is built upon Boyer-Moore, but utilized the "Pigeonhole principle" to bridge the gap between exact matching and approximate matching. It is more robust in real life usage because it can allow a number of mismatches in each alignment hit.  
- Next, a different approximate matching algorithm was implemented, utilizing the 'Index class' instead of the 'Boyer-Moore' approach. Both utilized the 'pigeonhole principle'


In [19]:
# First, we import a module authored by Ben Langmead, which defines functions to preprocess sequence 
# for the Boyer-Moore matching
from bm_preproc import BoyerMoore
import bm_preproc

### 1. Define naive matching algorithm with counts
-Additionally returns number of alignments and character comparisons (for efficiency comparison)

In [2]:
def naive_with_counts(p, t):
    '''naive exact matching algorithm, p=pattern, t=text,
    returns indexes of matches, and  
    number of alignments and character comparisons '''
    occurrences = []
    alignments = 0
    char_comparisons = 0
    for i in range(len(t)-len(p)+1):
        match = True
        alignments += 1
        for j in range(len(p)):
            char_comparisons += 1
            if t[i+j] != p[j]: 
                match = False
                break
        if match:
            occurrences.append(i)
    return occurrences, alignments, char_comparisons

### 2. Define Boyer-Moore matching algorithm with counts

In [3]:
def boyer_moore_with_counts(p, p_bm, t):
    '''Boyer-Moore matching, p=pattern, p_bm=Boyer-Moore object for p, t=text,
    returns indexes of matches, and  
    number of alignments and character comparisons '''
    i = 0 
    alignments = 0
    char_comparisons = 0
    occurrences = []
    while i < len(t) - len(p) + 1:
        alignments += 1
        shift = 1
        mismatch = False
        for j in range(len(p)-1, -1, -1): # move from right to left
            char_comparisons += 1
            if p[j] != t[i+j]: # if mismatch
                skip_bc = p_bm.bad_character_rule(j, t[i+j]) # apply bad character rule
                skip_gs = p_bm.good_suffix_rule(j) # apply good suffix rule
                shift = max(skip_bc, skip_gs, shift) # skip comparisons
                mismatch = True
                break
            
        if not mismatch: # if mismatch = False
            occurrences.append(i)
            skip = p_bm.match_skip()
            shift = max(skip, shift)

        i += shift
    return occurrences, alignments, char_comparisons

        
    

##### Test case 1 

In [4]:
# Naive 
p = 'word'
t = 'there would have been a time for such a word'
occ, num1, num2 = naive_with_counts(p, t)

print('Naive matching:', occ, num1, num2)

Naive matching: [40] 41 46


In [8]:
# Boyer-Moore
p = 'word'
t = 'there would have been a time for such a word'
lowercase_alphabet = 'abcdefghijklmnopqrstuvwxyz '
p_bm = BoyerMoore(p, lowercase_alphabet)
occ, num1, num2= boyer_moore_with_counts(p, p_bm, t)

print('Boyer-Moore:', occ, num1, num2)

Boyer-Moore: [40] 12 15


##### Test case 2

In [9]:
# Naive 
p = 'needle'
t = 'needle need noodle needle'
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, t)
print(occurrences, num_alignments, num_character_comparisons)

[0, 19] 20 35


In [10]:
# Boyer-Moore
p = 'needle'
t = 'needle need noodle needle'
p_bm = BoyerMoore(p, lowercase_alphabet)
occ2, num_bm_1, num_bm_2 = boyer_moore_with_counts(p, p_bm, t)
print('Boyer-Moore:', occ2, num_bm_1, num_bm_2)

Boyer-Moore: [0, 19] 5 18


##### Test case 3 using human genome chr1

In [1]:
from functions.parse_genome import parse_fasta
# parse human genome data chr1
human = parse_fasta('data/chr1.GRCh38.excerpt.fasta')

In [21]:
# Naive
p = 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'
t = human
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, t)
print(occurrences, num_alignments, num_character_comparisons)

[56922] 799954 984143


In [23]:
p = 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'
t = human
p_bm = BoyerMoore(p, alphabet='ACGT')
occ2, num_bm_1, num_bm_2 = boyer_moore_with_counts(p, p_bm, t)
print('Boyer-Moore:', occ2, num_bm_1, num_bm_2)

Boyer-Moore: [56922] 127974 165191


### 3. Approximate matching with k-mer index vs. Boyer-Moore, both applying pigeonhole principle

First, write function to apply approximate matching combining Boyer-Moore and pigeonhole principle

In [27]:
def approximate_match(p, t, n):
    '''Apply pigeonhole principle to do approximate matching
    n = number of mismatches allowed'''
    segment_length = int(round(len(p)/(n+1))) # divide to n+1 segments
    all_matches = set()
    
    for i in range(n+1): # For each segment, look for matches
        start = i*segment_length # the start location of the segment
        end = min(len(p), (i+1)*segment_length)# last seg may not be multiple of n
        p_bm = BoyerMoore(p[start:end], alphabet='ACGT') # create look-up table
        matches = boyer_moore(p[start:end], p_bm, t)
        #print(i, start, end,matches)
        
        for m in matches: # For each match in t, check the rest of p and see how many mismatches are there
            # NOTE: m are locations in t!
            
            if m < start or m-start+len(p)>len(t): # be sure m does not run off the start or end of t
                continue
                
            mismatches = 0
            # check for mismatches on the left and right side of the segment
            # we are essentially compare the whole p, but no need to compare the segment again
            # so the slicing in t is the same for the left and right comparisons, as t[m-start+j]
            # with j in range(0, start) for left and in range(end, len(p)) for the right. 
            for j in range(0,start): # check left side of the segment in p
                print('left, j=',j,'p[j]',p[j],'t[m-start+j]',t[m-start+j])
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n: # remember to break to be efficient
                        break
            
            for j in range(end,len(p)): # check right side 
                print('right, j=',j,'p[j]',p[j],'t[m-start+j]',t[m-start+j])
                if not p[j] == t[m-start+j]: # NOTE: the slicing of t is the same as left side
        
                    mismatches += 1
                    if mismatches > n:
                        break
            
            if mismatches <= n:
                all_matches.add(m-start) # NOTE: use '.add', not '.append' for set()
                # NOTE: use (m-start) to store locations that match the start of p, not the match at the ith segment of p. 
        
    return list(all_matches) # convert to list
        
        

Next, implement approximate matching differently, use Index method.

Previsouly with pigeonhole principle, we divide p into n+1 segments and find exact matching in one of the segments (while allowing mismatches in the others).

With the Index method, we firstly create a list of kmers for the text t, then search the first kmer of p (can be other kmers of p as well) against the kmers list of t. Record the hits, the compare the rest of p with t at every hit, allowing up to n mismatches. 


Import the K-mer index class

In [2]:
# %load functions/kmer_index.py
#!/usr/bin/env python

"""kmer_index.py: A k-mer index for indexing a text."""

__author__ = "Ben Langmead"

import bisect


class Index(object):
    """ Holds a substring index for a text T """

    def __init__(self, t, k):
        """ Create index from all substrings of t of length k """
        self.k = k  # k-mer length (k)
        self.index = []
        for i in range(len(t) - k + 1):  # for each k-mer
            self.index.append((t[i:i+k], i))  # add (k-mer, offset) pair
        self.index.sort()  # alphabetize by k-mer

    def query(self, p):
        """ Return index hits for first k-mer of p """
        kmer = p[:self.k]  # query with first k-mer
        i = bisect.bisect_left(self.index, (kmer, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != kmer:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits


In [32]:
# Approx matching algorithm
def approx_match_index(p, t, k, n):
    '''Function to find match of p in t, with up to n mismatches
    use Index Class to create kmers of length k and verify hits'''
    myindex = Index(t,k)
    occurrences = set()
    seg_length = int(round(len(p)/(n+1))) # p = 24
    index_hits = 0 # count index hits
    for seg in range(n+1): #number of 'pigeonholes' is n+1
        start = seg*seg_length
        end = (seg+1)*seg_length

        for i in myindex.query(p[start:end]):
            index_hits += 1
            mismatches = 0
            for j in range(0, start):
                if p[j] != t[i-start+j]:
                    mismatches += 1
                    #print(i, p[j], t[i-start+j],mismatches)
                    if mismatches > n:
                        break
            for j in range(end, len(p)):
                if p[j] != t[i-start+j]:
                    mismatches += 1
                    #print(i, p[j], t[i-start+j],mismatches)
                    if mismatches > n:
                        break                    
            
            if mismatches <= n:
                occurrences.add(i-start)
                
    return list(occurrences), index_hits

In [33]:
# Test case 1
p = 'GGCGCGGTGGCTCACGCCTGTAAT'
t = human 
n= 2
occ, index_hits = approx_match_index(p,t,k=8,n=2)
print(occ)

[84641, 160162, 724927, 273669, 147558, 364263, 421221, 681737, 717706, 465647, 429299, 747359, 657496, 160729, 56922, 635931, 191452, 551134, 262042]


In [34]:
print(index_hits)

90


In [66]:
# Compaire with naive exact match
def naive_n_mismatch(t,p,n):
    occurrences = []
    
    for i in range(len(t)-len(p)+1):
        match = True
        mismatch = 0 
        for j in range(len(p)):

            if (t[i+j] != p[j]): 
                mismatch += 1
                if mismatch > n:
                    match = False
                    break
        if match:
            occurrences.append(i)
    return occurrences


In [197]:
# Test case 1
p = 'GGCGCGGTGGCTCACGCCTGTAAT'
x = 'GGTGCGGTGGCTCATGCCTGTAAT'
t = human 
n= 2
occ1 = naive_n_mismatch(t,p,n)
print(occ1)

[56922, 84641, 147558, 160162, 160729, 191452, 262042, 273669, 364263, 421221, 429299, 465647, 551134, 635931, 657496, 681737, 717706, 724927, 747359]


Use a different version (more general) of indexing method which allows alternative slicing 

In [55]:
# Define the general indexing class which allows slicing with intervals >0
import bisect
   
class SubseqIndex(object):
    """ Holds a subsequence index for a text T """
    
    def __init__(self, t, k, ival):
        """ Create index from all subsequences consisting of k characters
            spaced ival positions apart.  E.g., SubseqIndex("ATAT", 2, 2)
            extracts ("AA", 0) and ("TT", 1). """
        self.k = k  # num characters per subsequence extracted
        self.ival = ival  # space between them; 1=adjacent, 2=every other, etc
        self.index = []
        self.span = 1 + ival * (k - 1)
        for i in range(len(t) - self.span + 1):  # for each subseq
            self.index.append((t[i:i+self.span:ival], i))  # add (subseq, offset)
        self.index.sort()  # alphabetize by subseq
    
    def query(self, p):
        """ Return index hits for first subseq of p """
        subseq = p[:self.span:self.ival]  # query with first subseq
        i = bisect.bisect_left(self.index, (subseq, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != subseq:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits


Approximate matching using subsequence (e.g., slice every other character)

In [62]:
def approx_match_SubseqIndex(p, t, k, ival, n):
    '''Function to find match of p in t, with up to n mismatches
    use Index Class to create kmers of length k and verify hits'''
    myindex = SubseqIndex(t,k,ival)
    occurrences = set()
    seg_length = int(round(len(p)/(n+1))) # p = 24
    span = 1 + ival*(k-1)
    index_hits = 0 # count index hits
    
    for seg in range(n+1): #number of 'pigeonholes' is n+1
        
        for i in myindex.query(p[seg:]): # for each match of the specific subsequence in p (n+1 total)
            index_hits += 1
            mismatches = 0
            # compare whole sequence of p with t 
            # I could try remove the already matched subsequence from comparisions, but the code can get a lot more complicated
            for j in range(len(p)):
                if p[j] != t[i-seg+j]:
                    mismatches += 1
                    #print(i, p[j], t[i-start+j],mismatches)
                    if mismatches > n:
                        break                  
            
            if mismatches <= n:
                occurrences.add(i-seg)
                
    return list(occurrences), index_hits

In [63]:
# Test case 1
p = 'GGCGCGGTGGCTCACGCCTGTAAT'
t = human 
n= 2
occ, index_hits = approx_match_SubseqIndex(p,t,k=8,ival=3, n=2)
print(occ)

[84641, 160162, 724927, 273669, 147558, 364263, 421221, 681737, 717706, 465647, 429299, 747359, 657496, 160729, 56922, 635931, 191452, 551134, 262042]


In [64]:
print(len(occ))

19


In [67]:
print(index_hits)  #Fewer index hits, indicating higher specificity

79
